In [1]:
import math 
import random

In [2]:
# math functions used for RSA algorithm

In [3]:
def gcd(x, y):  
    while(y):
     x, y = y, x % y  
    return x

In [4]:
def lcm(a, b):
    return abs(a*b) // math.gcd(a, b)

In [5]:
def phi(p,q):
    return lcm((p-1),(q-1))

In [6]:
def encript(e,n,m):
    return pow(m,e,n) 

In [7]:
def decript(d,n,c):
    return pow(c,d,n) 

In [8]:
def getN(p,q):
    return p*q

In [9]:
def getE(phi):
    while (True):
        e = random.randrange(2, phi)
        if (gcd(e, phi) == 1):
            return e    

In [10]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modInverse(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m


In [11]:
def getD(e,n):
    return modInverse(e,n)

In [12]:
def stringbuilder(s):  
    new = ""
    for x in s:
        new += x 
    return new

In [13]:
# primeNumberGen functions

In [14]:
# legendre symbol (a|m)
# note: returns m-1 if a is a non-residue, instead of -1
def legendre(a, m):
  return pow(a, (m-1) >> 1, m)

# strong probable prime
def is_sprp(n, b=2):
  d = n-1
  s = 0
  while d&1 == 0:
    s += 1
    d >>= 1

  x = pow(b, d, n)
  if x == 1 or x == n-1:
    return True

  for r in range(1, s):
    x = (x * x)%n
    if x == 1:
      return False
    elif x == n-1:
      return True

  return False

# lucas probable prime
# assumes D = 1 (mod 4), (D|n) = -1
def is_lucas_prp(n, D):
  P = 1
  Q = (1-D) >> 2

  # n+1 = 2**r*s where s is odd
  s = n+1
  r = 0
  while s&1 == 0:
    r += 1
    s >>= 1

  # calculate the bit reversal of (odd) s
  # e.g. 19 (10011) <=> 25 (11001)
  t = 0
  while s > 0:
    if s&1:
      t += 1
      s -= 1
    else:
      t <<= 1
      s >>= 1

  # use the same bit reversal process to calculate the sth Lucas number
  # keep track of q = Q**n as we go
  U = 0
  V = 2
  q = 1
  # mod_inv(2, n)
  inv_2 = (n+1) >> 1
  while t > 0:
    if t&1 == 1:
      # U, V of n+1
      U, V = ((U + V) * inv_2)%n, ((D*U + V) * inv_2)%n
      q = (q * Q)%n
      t -= 1
    else:
      # U, V of n*2
      U, V = (U * V)%n, (V * V - 2 * q)%n
      q = (q * q)%n
      t >>= 1

  # double s until we have the 2**r*sth Lucas number
  while r > 0:
      U, V = (U * V)%n, (V * V - 2 * q)%n
      q = (q * q)%n
      r -= 1

  # primality check
  # if n is prime, n divides the n+1st Lucas number, given the assumptions
  return U == 0

# primes less than 212
small_primes = set([
    2,  3,  5,  7, 11, 13, 17, 19, 23, 29,
   31, 37, 41, 43, 47, 53, 59, 61, 67, 71,
   73, 79, 83, 89, 97,101,103,107,109,113,
  127,131,137,139,149,151,157,163,167,173,
  179,181,191,193,197,199,211])

# pre-calced sieve of eratosthenes for n = 2, 3, 5, 7
indices = [
    1, 11, 13, 17, 19, 23, 29, 31, 37, 41,
   43, 47, 53, 59, 61, 67, 71, 73, 79, 83,
   89, 97,101,103,107,109,113,121,127,131,
  137,139,143,149,151,157,163,167,169,173,
  179,181,187,191,193,197,199,209]

# distances between sieve values
offsets = [
  10, 2, 4, 2, 4, 6, 2, 6, 4, 2, 4, 6,
   6, 2, 6, 4, 2, 6, 4, 6, 8, 4, 2, 4,
   2, 4, 8, 6, 4, 6, 2, 4, 6, 2, 6, 6,
   4, 2, 4, 6, 2, 6, 4, 2, 4, 2,10, 2]

max_int = 2147483647

# an 'almost certain' primality check
def is_prime(n):
  if n < 212:
    return n in small_primes

  for p in small_primes:
    if n%p == 0:
      return False

  # if n is a 32-bit integer, perform full trial division
  if n <= max_int:
    i = 211
    while i*i < n:
      for o in offsets:
        i += o
        if n%i == 0:
          return False
    return True

  # Baillie-PSW
  # this is technically a probabalistic test, but there are no known pseudoprimes
  if not is_sprp(n): return False
  a = 5
  s = 2
  while legendre(a, n) != n-1:
    s = -s
    a = s-a
  return is_lucas_prp(n, a)

# next prime strictly larger than n
def next_prime(n):
  if n < 2:
    return 2
  # first odd larger than n
  n = (n + 1) | 1
  if n < 212:
    while True:
      if n in small_primes:
        return n
      n += 2

  # find our position in the sieve rotation via binary search
  x = int(n%210)
  s = 0
  e = 47
  m = 24
  while m != e:
    if indices[m] < x:
      s = m
      m = (s + e + 1) >> 1
    else:
      e = m
      m = (s + e) >> 1

  i = int(n + (indices[m] - x))
  # adjust offsets
  offs = offsets[m:]+offsets[:m]
  while True:
    for o in offs:
      if is_prime(i):
        return i
      i += o

In [15]:
#key gen for RSA

In [27]:
2**2048

32317006071311007300714876688669951960444102669715484032130345427524655138867890893197201411522913463688717960921898019494119559150490921095088152386448283120630877367300996091750197750389652106796057638384067568276792218642619756161838094338476170470581645852036305042887575891541065808607552399123930385521914333389668342420684974786564569494856176035326322058077805659331026192708460314150258592864177116725943603718461857357598351152301645904403697613233287231227125684710820209725157101726931323469678542580656697935045997268352998638215525166389437335543602135433229604645318478604952148193555853611059596230656

In [32]:
print("Defining p")
p=next_prime(random.randrange((10**311), (10**619)))
print("p =" ,p)
print("Defining q")
q=next_prime(random.randrange((10**311), (10**619)))
if(p==q):
 q = next_prime(q)
print("q =" ,q)
print("Defining n = pq")
n= getN(p,q)
print("n =" ,n)
print("Defining λ(n)")
f= phi(p,q)
print("λ(n) =" ,f)
print("Defining e")
e= getE(f)
print("e =" ,e)
print("Defining d")
d= getD(e,f)
print("d =" ,d)

Defining p
p = 5993795944818791462685609185333792185385030725680648292970570218228531060955031321436353162972953419382427681900228446372365495113745515420923643825522582519209649474092993551142150628631846561866830195582422386975192013783986443043951502857882126020026494400916554508446431482176647932973292371436919434224313965544781725868990467258074597657624938749394054340683227719630482621726620869001684806964060527833646090664664244787914317166416429069738889216226353629440234005514344164021379226428582191401147651412511786927295838287614929136314771663022128535454182323483594664625897941077722739790048951837184075227573769
Defining q
q = 73335315975426704058982921241331156920624910217314433179996977416578295932624766775203447468659483737039766451326840104295338687759883545262352321343637358528040294109053512028313128271624380408680990485823822016460379625739111959996463659042573878629152012408682473097315531380266255209211049514884784986398516799799496992225773157972658284194821164

In [33]:
msg = "Vex launches a wave of mist that deals [60/110/160/210/260 (+60% AP)] magic damage. After a delay, the wave becomes smaller and faster. "

In [ ]:
k = [encript(e,n,ord(m)) for m in msg]

In [ ]:
k

In [ ]:
k2 = [decript(d,n,c) for c in k]

In [ ]:
msg2 = [chr(c) for c in k2]

In [ ]:
msg3 =  stringbuilder(msg2)

In [ ]:
print(msg3 == msg)

In [ ]:
p==q